# Variant-Calling Oxford Nanopore Tech (ONT) Long-Read Whole Genome Sequencing (WGS) Data

**GOAL**: *create a model to distinguish true variants from artifactual variant calls*

- data is from the [Nanopore WGS Consortium](https://github.com/nanopore-wgs-consortium/NA12878/blob/master/Genome.md) GitHub repository
- they sequenced the CEPH1463 (NA12878/GM12878, Ceph/Utah pedigree) human genome reference standard on the Oxford Nanopore MinION using 1D ligation kits (450 bp/s) and R9.4 chemistry (FLO-MIN106)
- sequencing was done by [5 labs](https://www.nature.com/articles/nbt.4060) on 53 flow cells using both DNA purchased from Coriell as well as DNA extracted from cultured cells


This analysis involved:
- downloading raw FAST5 files (~4.5TB) and basecalling to FASTQ with the latest, [high-performance](https://aws.amazon.com/blogs/hpc/benchmarking-the-oxford-nanopore-technologies-basecallers-on-aws/)  [dorado basecaller](https://github.com/nanoporetech/dorado) using the 'sup' (highest accuracy) model 
- mapping to the hg38 human reference genome to generate BAM files (minimap2, samtools)
- marking duplicates (samtools)
- merging the resulting 52 BAM files into 1 file (samtools)
- variant calling to generate VCF file (bcftools mpileup & call)
- intersecting with gold-standard [NIST Genome in a Bottle](https://www.nist.gov/programs-projects/genome-bottle) data to enable comparison (bedtools intersect)
- creating machine learning and deep learning models to distinguish true variant calls from artifacts

Analysis was performed on a high performance computer (HPC) cluster running CentOS Linux with SLURM as a resource manager & job scheduler.

## 1. Mapping to hg38 human reference genome (minimap2, samtools)

We can use a simple bash script to loop over all the FASTQ files in a directory and submit them as batch jobs to the HPC using the SLURM scheduler

In [1]:
!cat ~/tools/slurm_scripts/bash_slurm_sbatch_loop_directory_files_fastq_221221.sh

#!/bin/bash
# loop over all *.fastq files in the directory input_dir
# print the filename (so we have some visual progress indicator)
# then submit to SLURM


slurm_script=$1        # SLURM SCRIPT OF INTEREST
input_dir=$2             # DIRECTORY OF INTEREST
output_dir=$3          # OUTPUT DIRECTORY

for FILE in "${input_dir}"*.fastq; do
echo ${FILE}
sbatch $slurm_script \
    ${FILE} \
    $output_dir
sleep 1 # pause to be kind to the scheduler
done

---
In this case, the SLURM job will be mapping each of the 52 FASTQ files to the hg38 human reference genome using minimap2, the sequence aligner recommended by Oxford Nanopore Technologies. To output a sorted BAM file, samtools is used on the output of minimap2.

In [5]:
!cat ~/tools/slurm_scripts/minimap2_samtoolsBam_RG_231112_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=minimap2_samtoolsBam_RG_231112
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/minimap2_samtoolsBam_RG_231112_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/minimap2_samtoolsBam_RG_231112_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate minimap2_224                      ### CONDA ENVIRONMENT NAME
conda info
echo ""

call="minimap2 --version" # to get the version
echo $call
eval $call
echo ""

call="samtools --version" # to get the version
echo $call
eval $call
echo ""

call="bcftools --version" # to get the version
echo $call
eval $call
echo ""

fastq_path=$1
bam_output_dir=$2

flowcell_asic_id=$(echo $fastq_path | awk -F'[/_.]' '{print $10}')    # https://github.com/nan

---
And here is the actual job submission to SLURM:

In [6]:
# submit fastq-to-bam jobs to slurm
!mkdir -p 01_bam_raw

!bash ~/tools/slurm_scripts/bash_slurm_sbatch_loop_directory_files_fastq_221221.sh \
    ~/tools/slurm_scripts/minimap2_samtoolsBam_RG_231112_slurm.sh \
    ~/nanopore/rogstrix/dorado/00_fastq_raw/ \
    01_bam_raw/

!squeue -u fmbuga

/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB39043-3709921973.fastq
Submitted batch job 1111239
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB39075-4246400039.fastq
Submitted batch job 1111240
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB39088-288418386.fastq
Submitted batch job 1111241
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB41174-3976885577.fastq
Submitted batch job 1111242
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB42205-3573838535.fastq
Submitted batch job 1111243
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB42260-4177064552.fastq
Submitted batch job 1111244
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB42316-216722908.fastq
Submitted batch job 1111245
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB42395-4178605061.fastq
Submitted batch job 1111246
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB42451-4239353418.fastq
Submitted batch job 1111247
/home/fmbuga/nanopore/rogstrix/dorado/00_fastq_raw/FAB424

In [8]:
# check the output and confirm all 52 files were processed
!ls -1 01_bam_raw/*.bam \
    | tee >(wc -l)

01_bam_raw/FAB39043-3709921973.bam
01_bam_raw/FAB39075-4246400039.bam
01_bam_raw/FAB39088-288418386.bam
01_bam_raw/FAB41174-3976885577.bam
01_bam_raw/FAB42205-3573838535.bam
01_bam_raw/FAB42260-4177064552.bam
01_bam_raw/FAB42316-216722908.bam
01_bam_raw/FAB42395-4178605061.bam
01_bam_raw/FAB42451-4239353418.bam
01_bam_raw/FAB42473-4179682758.bam
01_bam_raw/FAB42476-3843483077.bam
01_bam_raw/FAB42561-356443753.bam
01_bam_raw/FAB42704-87746129.bam
01_bam_raw/FAB42706-4111103328.bam
01_bam_raw/FAB42798-3306352129.bam
01_bam_raw/FAB42804-84744914.bam
01_bam_raw/FAB42810-352384898.bam
01_bam_raw/FAB42828-288548394.bam
01_bam_raw/FAB43577-3574887596.bam
01_bam_raw/FAB44989-2567311907.bam
01_bam_raw/FAB45271-152889212.bam
01_bam_raw/FAB45277-86567043.bam
01_bam_raw/FAB45280-222619780.bam
01_bam_raw/FAB45321-19064779.bam
01_bam_raw/FAB45321-285174896.bam
01_bam_raw/FAB45332-551111640.bam
01_bam_raw/FAB46664-288286712.bam
01_bam_raw/FAB46683-4246923067.bam
01_bam_raw/FAB49164-4045668814.bam
01_

In [9]:
# check how mapping went
!samtools flagstats \
    --threads 28 \
    --verbosity 5 \
    01_bam_raw/FAF01132-84868110.bam | head -n 8

1650805 + 0 in total (QC-passed reads + QC-failed reads)
719983 + 0 primary
480392 + 0 secondary
450430 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
1588422 + 0 mapped (96.22% : N/A)
657600 + 0 primary mapped (91.34% : N/A)


## 2. Marking duplicates (samtools)

Next, we [mark duplicates](http://www.htslib.org/doc/samtools-markdup.html) on the 52 BAM files using `samtools markdup`.

We can re-purpose the looping bash script used above in the mapping step for SLURM job submission with a slight change so that it loops over all BAM files in a folder instead of all FASTQ files

In [1]:
!cat ~/tools/slurm_scripts/bash_slurm_sbatch_loop_directory_files_samtools_markdup_221222.sh

#!/bin/bash
# loop over all *.bam files in the directory input_dir
# print the filename (so we have some visual progress indicator)
# then submit to SLURM


slurm_script=$1        # SLURM SCRIPT OF INTEREST
input_dir=$2             # DIRECTORY OF INTEREST                   
output_dir=$3             # OUTPUT DIRECTORY                   


for FILE in "${input_dir}"*.bam; do
echo ${FILE}
sbatch $slurm_script \
    ${FILE} \
    $output_dir
sleep 1 # pause to be kind to the scheduler
done

---
In order to use `samtools markdup` to mark duplicates in the BAM files, the reads first have to be [ordered by name](http://www.htslib.org/doc/samtools-markdup.html) (`samtools collate`), annotated with [tags for mate score and mate coordinates](http://www.htslib.org/doc/samtools-fixmate.html) (`samtools fixmate`) and [sorted in position order](http://www.htslib.org/doc/samtools-sort.html) (`samtools sort`). These commands can be chained with pipes with the BAM files going in and outputting processed BAM files with duplicates marked.

In [2]:
# the SLURM job script for marking duplicates
!cat ~/tools/slurm_scripts/samtools_collate_fixmate_sort_markdup_221222_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=samtools_collate_fixmate_sort_markdup_221222
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/samtools_collate_fixmate_sort_markdup_221222_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/samtools_collate_fixmate_sort_markdup_221222_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate minimap2_224                      ### CONDA ENVIRONMENT NAME
conda info
echo ""

call="samtools --version" # to get the version
echo $call
eval $call
echo ""

input_bam=$1
output_dir=$2

out_name=$(basename $input_bam .bam)

call=$(samtools collate \
    -Ou \
    --threads 28 \
    $input_bam | \
        samtools fixmate \
            - \
            - \
            --threads 28 \
            -u 

---
And the SLURM job submission with the looping bash script

In [3]:
!mkdir -p 02_bam_markdup
!bash ~/tools/slurm_scripts/bash_slurm_sbatch_loop_directory_files_samtools_markdup_221222.sh \
    ~/tools/slurm_scripts/samtools_collate_fixmate_sort_markdup_221222_slurm.sh \
    01_bam_raw/ \
    02_bam_markdup/

!squeue -u fmbuga

01_bam_raw/FAB39043-3709921973.bam
Submitted batch job 1111379
01_bam_raw/FAB39075-4246400039.bam
Submitted batch job 1111380
01_bam_raw/FAB39088-288418386.bam
Submitted batch job 1111381
01_bam_raw/FAB41174-3976885577.bam
Submitted batch job 1111382
01_bam_raw/FAB42205-3573838535.bam
Submitted batch job 1111383
01_bam_raw/FAB42260-4177064552.bam
Submitted batch job 1111384
01_bam_raw/FAB42316-216722908.bam
Submitted batch job 1111385
01_bam_raw/FAB42395-4178605061.bam
Submitted batch job 1111386
01_bam_raw/FAB42451-4239353418.bam
Submitted batch job 1111387
01_bam_raw/FAB42473-4179682758.bam
Submitted batch job 1111388
01_bam_raw/FAB42476-3843483077.bam
Submitted batch job 1111389
01_bam_raw/FAB42561-356443753.bam
Submitted batch job 1111390
01_bam_raw/FAB42704-87746129.bam
Submitted batch job 1111391
01_bam_raw/FAB42706-4111103328.bam
Submitted batch job 1111392
01_bam_raw/FAB42798-3306352129.bam
Submitted batch job 1111393
01_bam_raw/FAB42804-84744914.bam
Submitted batch job 1111394

---
Let's have a look at one of the BAM files before and after marking duplicates using `samtools flagstats`

In [4]:
!samtools flagstats \
    --threads 28 \
    --verbosity 5 \
    01_bam_raw/FAF01132-84868110.bam | head -n 8

1650805 + 0 in total (QC-passed reads + QC-failed reads)
719983 + 0 primary
480392 + 0 secondary
450430 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
1588422 + 0 mapped (96.22% : N/A)
657600 + 0 primary mapped (91.34% : N/A)


In [5]:
!samtools flagstats \
    --threads 28 \
    --verbosity 5 \
    02_bam_markdup/FAF01132-84868110_collate_fixmate_sort_markdup.bam | head -n 8

1650805 + 0 in total (QC-passed reads + QC-failed reads)
719983 + 0 primary
480392 + 0 secondary
450430 + 0 supplementary
19503 + 0 duplicates
19503 + 0 primary duplicates
1588422 + 0 mapped (96.22% : N/A)
657600 + 0 primary mapped (91.34% : N/A)


We see that for this BAM file, 19,503 of the 1,650,805 have been marked as duplicates (~1.2%)

## 3. Merging the resulting 52 BAM files into 1 file (samtools)

Next, we use `samtools merge` to [combine the 52 BAM files into one](http://www.htslib.org/doc/samtools-merge.html).

In [2]:
# the SLURM job script for merging all the BAM files in the folder
!cat ~/tools/slurm_scripts/samtools_merge_221128_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=samtools_merge_221128
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/samtools_merge_221128_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/samtools_merge_221128_%j.err
#
#SBATCH --ntasks=8
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G


datenow=$(date)
echo $datenow
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate minimap2_224                      ### CONDA ENVIRONMENT NAME
conda info
echo ""

call="samtools --version"
echo $call
eval $call
echo ""

out_bam=$1
in_bams_dir=$2

# samtools merge

call=$(samtools merge \
        $out_bam \
        "${in_bams_dir}"*.bam \
        --threads 224 \
        --write-index \
        --verbosity 10) # INPUT & OUTPUT BAMS
    
echo $call
eval $call
echo ""

echo "samtools merge COMPLETE."
echo ""
    
##################


echo ""
end=$(date +%s)
echo "

---
And here's the SLURM job submission:

In [3]:
!mkdir -p 03_bam_merged
!sbatch ~/tools/slurm_scripts/samtools_merge_221128_slurm.sh \
    03_bam_merged/dorado.bam \
    02_bam_markdup/

!sleep 10
!squeue -u fmbuga

Submitted batch job 1111544
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1111543     nodes     bash   fmbuga  R       9:35      1 node05
           1111544     nodes samtools   fmbuga  R       0:10      8 node[40-47]


---
Let's see what the stats look like for our final mega BAM file:

In [4]:
!samtools flagstats \
    --threads 28 \
    03_bam_merged/dorado.bam | head -n 8

28256836 + 0 in total (QC-passed reads + QC-failed reads)
14685190 + 0 primary
7283682 + 0 secondary
6287964 + 0 supplementary
207320 + 0 duplicates
207320 + 0 primary duplicates
26819340 + 0 mapped (94.91% : N/A)
13247694 + 0 primary mapped (90.21% : N/A)


So about 0.7% of the reads are duplicates

## 4. Calling variants (bcftools)

We use `bcftools mpileup` to [generate genotype likelihoods](https://samtools.github.io/bcftools/bcftools.html#mpileup) and `bcftools call` to [make the actual variant call](https://samtools.github.io/bcftools/bcftools.html#call). The prior for the expected substitution rate (`bcftools call` parameter -P) is set to a very high value to [make the calling as sensitive as possible](https://samtools.github.io/bcftools/howtos/variant-calling.html) to minimize false negatives with the hope that the model we build will filter out the majority of the false positives that result.

In [5]:
# SLURM job script to do the variant calling
!cat ~/tools/slurm_scripts/bcftools_mpileup_call_ONT_P099_minMQ0_231114_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=bcftools_mpileup_call_ONT_P099_minMQ0_231114
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_mpileup_call_ONT_P099_minMQ0_231114_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_mpileup_call_ONT_P099_minMQ0_231114_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=120G


# "...multithreading ...is currently used only for the compression of the output stream, only when --output-type is b or z"
# https://samtools.github.io/bcftools/bcftools.html
# "Use the -Ou option when piping between bcftools subcommands to speed up performance by removing unnecessary compression/decompression and VCF←→BCF conversion"
# for ONT https://github.com/samtools/bcftools/issues/1666

# "For ONT it may be beneficial to also run bcftools call with a higher -P, eg -P0.01 or -P 0.1"


datenow=$(date)
echo $datenow
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo "hostname: $HOST

In [6]:
# submitting the variant calling job to SLURM
!mkdir -p 04_vcf_raw
!sbatch ~/tools/slurm_scripts/bcftools_mpileup_call_ONT_P099_minMQ0_231114_slurm.sh \
    03_bam_merged/dorado.bam \
    04_vcf_raw/

!sleep 5
!squeue -u fmbuga

Submitted batch job 1111545
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1111543     nodes     bash   fmbuga  R    1:02:38      1 node05
           1111545     nodes bcftools   fmbuga  R       0:05      1 node14


---
Let's have a look at the output VCF using `bcftools stats`:

In [2]:
# check stats of the VCF file
!bcftools stats \
    --threads 28 \
    04_vcf_raw/dorado_bcftools_P_099_minMQ_0.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	6484598
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	6484598
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	16587
SN	0	number of multiallelic SNP sites:	16587


Over 6 million SNPs were called. Considering humans on average only have about 1 SNP per 1000 base pairs, we expect about half of these calls to be false positives since the human genome has ~3.2 billion base pairs. Note that indel calling is turned off which is why 0 indels are recorded.

We can compare it to the number of SNPs in the GIAB truth-set

In [3]:
# check stats of the GIAB VCF file
!bcftools stats \
    --threads 28 \
    ~/tools/GIAB/GRCh38/HG001_NA12878/v4_2_1/HG001_GRCh38_v4_2_1.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	3893341
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	3358502
SN	0	number of MNPs:	0
SN	0	number of indels:	536489
SN	0	number of others:	90
SN	0	number of multiallelic sites:	42393
SN	0	number of multiallelic SNP sites:	1033


## 5. Intersect with Genome-in-a-Bottle variant calls (bedtools)

Next, we use `bedtools intersect` to overlap our variant callset with a BED file of the regions covered by NIST's Genome in a Bottle (GIAB) high-confidence variant calls of the same genome so we can compare our variant calls with a gold standard. Furthermore, the comparison will be what is used to create the labels (true variant vs artifact) for the model for artifactual variants.

In [4]:
# intersect dorado vcf with GIAB hi-conf BED
!mkdir -p 05_vcf_isec_GIAB
!bedtools intersect \
    -a 04_vcf_raw/dorado_bcftools_P_099_minMQ_0.vcf \
    -b ~/tools/GIAB/GRCh38/HG001_NA12878/v4_2_1/HG001_GRCh38_1_22_v4.2.1_benchmark.bed \
    -header > 05_vcf_isec_GIAB/dorado_bcftools_P_099_isec.vcf

***** WARNING: File 04_vcf_raw/dorado_bcftools_P_099_minMQ_0.vcf has inconsistent naming convention for record:
MAP2K3_chr17_22578583_22605165	151	.	A	G	189.708	.	DP=24;VDB=1.66594e-13;SGB=-0.692352;RPBZ=1.03992;MQBZ=-0.315869;MQSBZ=1.0942;BQBZ=-0.554502;SCBZ=1.11849;FS=0;MQ0F=0;AC=2;AN=2;DP4=1,0,13,8;MQ=57	GT:PL	1/1:187,40,0

***** WARNING: File 04_vcf_raw/dorado_bcftools_P_099_minMQ_0.vcf has inconsistent naming convention for record:
MAP2K3_chr17_22578583_22605165	151	.	A	G	189.708	.	DP=24;VDB=1.66594e-13;SGB=-0.692352;RPBZ=1.03992;MQBZ=-0.315869;MQSBZ=1.0942;BQBZ=-0.554502;SCBZ=1.11849;FS=0;MQ0F=0;AC=2;AN=2;DP4=1,0,13,8;MQ=57	GT:PL	1/1:187,40,0



In [5]:
# check stats of the isec VCF file
!bcftools stats \
    --threads 28 \
    05_vcf_isec_GIAB/dorado_bcftools_P_099_isec.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	3773914
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	3773914
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	4126
SN	0	number of multiallelic SNP sites:	4126


Filtering our VCF file to only look at the regions where NIST GIAB has high-confidence variant calls leaves us with ~3.8 million SNPs.

In [19]:
# filter GIAB VCF to keep SNPs only
!bcftools view \
    --include 'TYPE="snp"' \
    --threads 28 \
    ~/tools/GIAB/GRCh38/HG001_NA12878/v4_2_1/HG001_GRCh38_v4_2_1.vcf \
    > ~/tools/GIAB/GRCh38/HG001_NA12878/v4_2_1/HG001_GRCh38_v4_2_1_SNVs_ONLY.vcf

# check stats of the snps only VCF file
!bcftools stats \
    --threads 28 \
    ~/tools/GIAB/GRCh38/HG001_NA12878/v4_2_1/HG001_GRCh38_v4_2_1_SNVs_ONLY.vcf \
    | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	3356518
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	3356518
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	1033
SN	0	number of multiallelic SNP sites:	1033


### generate 3 VCFs (true positive, false positive, false negative)

In [32]:
# compress and index the two vcf files to allow reading by bcftools isec
!bgzip tmp_dir/dorado_sorted.vcf
!tabix -p vcf tmp_dir/dorado_sorted.vcf.gz
!bgzip tmp_dir/GIAB_sorted.vcf
!tabix -p vcf tmp_dir/GIAB_sorted.vcf.gz
# Create intersection and complements of two sets saving the output in dir/*
!bcftools isec \
    --threads 28 \
    tmp_dir/dorado_sorted.vcf.gz tmp_dir/GIAB_sorted.vcf.gz \
    -p tmp_dir

In [35]:
!cat tmp_dir/README.txt

This file was produced by vcfisec.
The command line was:	bcftools isec  --threads 28 -p tmp_dir tmp_dir/dorado_sorted.vcf.gz tmp_dir/GIAB_sorted.vcf.gz

Using the following file names:
tmp_dir/0000.vcf	for records private to	tmp_dir/dorado_sorted.vcf.gz
tmp_dir/0001.vcf	for records private to	tmp_dir/GIAB_sorted.vcf.gz
tmp_dir/0002.vcf	for records from tmp_dir/dorado_sorted.vcf.gz shared by both	tmp_dir/dorado_sorted.vcf.gz tmp_dir/GIAB_sorted.vcf.gz
tmp_dir/0003.vcf	for records from tmp_dir/GIAB_sorted.vcf.gz shared by both	tmp_dir/dorado_sorted.vcf.gz tmp_dir/GIAB_sorted.vcf.gz


In [6]:
# make new folder for 3 VCFs: TP, FP, FN (true positive, false positive, false negative)
!mkdir 06_vcf_TP_FP_FN

# copy the 3 vcfs to this folder
!cp tmp_dir/0000.vcf 06_vcf_TP_FP_FN/FP_snvs.vcf
!cp tmp_dir/0001.vcf 06_vcf_TP_FP_FN/FN_snvs.vcf
!cp tmp_dir/0002.vcf 06_vcf_TP_FP_FN/TP_snvs.vcf


In [7]:
# true positives (correct variants)
!bcftools stats \
    --threads 28 \
    06_vcf_TP_FP_FN/TP_snvs.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	3236774
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	3236774
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	926
SN	0	number of multiallelic SNP sites:	926


In [8]:
# false positives
!bcftools stats \
    --threads 28 \
    06_vcf_TP_FP_FN/FP_snvs.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	537140
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	537140
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	3200
SN	0	number of multiallelic SNP sites:	3200


In [9]:
# false negatives
!bcftools stats \
    --threads 28 \
    06_vcf_TP_FP_FN/FN_snvs.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	119744
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	119744
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	107
SN	0	number of multiallelic SNP sites:	107


---
### NEXT:
- generate features (201bps)
- generate labels (0 - not artifact, 1 - is artifact & 0 - TP, 1 - FP, 2 - FN): one-hot encode later in numpy
- pre-process for deep learning
- fit deep learning models